# F1 Prize Picks Optimizer

This notebook implements the Prize Picks optimization engine with:
- Kelly Criterion for optimal bet sizing
- Correlation management for multi-bet optimization
- Expected Value (EV) calculations
- Risk management and portfolio optimization
- Parlay and multi-leg bet optimization

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
# plt.style.use('seaborn-darkgrid') # Original style - may not work on all systems
# Safe plotting style setup
try:
    import seaborn as sns
    sns.set_theme()  # Modern seaborn initialization
except:
    try:
        plt.style.use('ggplot')  # Fallback style
    except:
        pass  # Use default style
sns.set_palette('husl')

In [ ]:
# Load necessary components
import sys
sys.path.append('.')
from f1db_data_loader import load_f1db_data

# Load data
print("Loading F1 data...")
f1_data = load_f1db_data(data_dir='../../data')

# Load saved models and feature store
try:
    # Load integrated model
    integrated_model = joblib.load('f1_integrated_evaluation_model.pkl')
    print("Loaded integrated evaluation model")
except:
    print("Integrated model not found - will use simulated predictions")
    integrated_model = None

# Load feature store
try:
    feature_store = pd.read_parquet('f1_feature_store.parquet')
    print(f"Loaded feature store with {len(feature_store)} records")
except:
    print("Feature store not found - will use basic features")
    feature_store = None

## 1. Prize Picks Bet Types and Structures

In [ ]:
class PrizePicksBetTypes:
    """
    Define Prize Picks bet types and payout structures
    """
    # Standard Prize Picks multipliers
    PAYOUTS = {
        2: 3.0,    # 2-pick entry pays 3x
        3: 6.0,    # 3-pick entry pays 6x
        4: 10.0,   # 4-pick entry pays 10x
        5: 20.0,   # 5-pick entry pays 20x
        6: 25.0    # 6-pick entry pays 25x
    }
    
    # Bet types available
    BET_TYPES = {
        'top_10': 'Will finish in top 10',
        'top_5': 'Will finish in top 5',
        'top_3': 'Will finish in top 3 (podium)',
        'points': 'Will score points',
        'h2h': 'Head-to-head matchup',
        'beat_teammate': 'Will beat teammate',
        'fastest_lap': 'Will set fastest lap',
        'grid_gain': 'Positions gained from start',
        'dnf': 'Will not finish (DNF)'
    }
    
    @staticmethod
    def calculate_payout(n_picks, stake=1.0):
        """Calculate potential payout for n picks"""
        if n_picks not in PrizePicksBetTypes.PAYOUTS:
            return 0
        return stake * PrizePicksBetTypes.PAYOUTS[n_picks]
    
    @staticmethod
    def required_win_rate(n_picks):
        """Calculate required win rate to break even"""
        if n_picks not in PrizePicksBetTypes.PAYOUTS:
            return 1.0
        return 1.0 / PrizePicksBetTypes.PAYOUTS[n_picks]

# Display Prize Picks structure
print("Prize Picks Payout Structure:")
print("=" * 40)
for picks, multiplier in PrizePicksBetTypes.PAYOUTS.items():
    breakeven = PrizePicksBetTypes.required_win_rate(picks)
    print(f"{picks} picks: {multiplier}x payout (breakeven: {breakeven:.1%})")

# Visualize payout structure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

picks = list(PrizePicksBetTypes.PAYOUTS.keys())
payouts = list(PrizePicksBetTypes.PAYOUTS.values())
breakeven_rates = [PrizePicksBetTypes.required_win_rate(p) for p in picks]

ax1.bar(picks, payouts, color='green', alpha=0.7)
ax1.set_xlabel('Number of Picks')
ax1.set_ylabel('Payout Multiplier')
ax1.set_title('Prize Picks Payout Structure')
ax1.grid(True, alpha=0.3)

ax2.plot(picks, breakeven_rates, 'o-', color='red', markersize=10)
ax2.axhline(y=0.5, color='gray', linestyle='--', label='50% win rate')
ax2.set_xlabel('Number of Picks')
ax2.set_ylabel('Required Win Rate')
ax2.set_title('Breakeven Win Rate by Picks')
ax2.set_ylim(0, 0.6)
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

## 2. Kelly Criterion Implementation

In [ ]:
class KellyCriterion:
    """
    Kelly Criterion for optimal bet sizing
    """
    def __init__(self, kelly_fraction=0.25):
        """
        Initialize with fractional Kelly (more conservative)
        kelly_fraction: Fraction of full Kelly to use (0.25 = quarter Kelly)
        """
        self.kelly_fraction = kelly_fraction
    
    def calculate_kelly_stake(self, probability, odds):
        """
        Calculate optimal stake using Kelly Criterion
        
        Args:
            probability: True probability of winning
            odds: Decimal odds (payout multiplier)
        
        Returns:
            Optimal fraction of bankroll to bet
        """
        if probability <= 0 or probability >= 1:
            return 0
        
        # Kelly formula: f = (p * o - 1) / (o - 1)
        # where p = probability, o = decimal odds
        q = 1 - probability  # Probability of losing
        
        kelly_full = (probability * odds - 1) / (odds - 1)
        
        # Apply fractional Kelly
        kelly_stake = kelly_full * self.kelly_fraction
        
        # Ensure non-negative and reasonable bounds
        return max(0, min(kelly_stake, 0.25))  # Max 25% of bankroll
    
    def calculate_multi_kelly(self, probabilities, n_picks):
        """
        Calculate Kelly stake for multi-pick parlays
        """
        # Combined probability (all must win)
        combined_prob = np.prod(probabilities)
        
        # Get Prize Picks payout
        payout = PrizePicksBetTypes.PAYOUTS.get(n_picks, 0)
        
        if payout == 0:
            return 0
        
        return self.calculate_kelly_stake(combined_prob, payout)
    
    def expected_value(self, probability, odds, stake=1.0):
        """
        Calculate expected value of a bet
        """
        win_amount = stake * (odds - 1)
        ev = probability * win_amount - (1 - probability) * stake
        return ev
    
    def calculate_growth_rate(self, probability, odds, kelly_stake):
        """
        Calculate expected growth rate using this stake size
        """
        if kelly_stake <= 0:
            return 0
        
        p = probability
        q = 1 - probability
        b = odds - 1
        f = kelly_stake
        
        growth_rate = p * np.log(1 + b * f) + q * np.log(1 - f)
        return growth_rate

# Demonstrate Kelly Criterion
kelly = KellyCriterion(kelly_fraction=0.25)

# Example calculations
probabilities = np.linspace(0.3, 0.8, 50)
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Kelly stake vs probability for different pick counts
ax = axes[0, 0]
for n_picks in [2, 3, 4, 5]:
    odds = PrizePicksBetTypes.PAYOUTS[n_picks]
    stakes = [kelly.calculate_kelly_stake(p, odds) for p in probabilities]
    ax.plot(probabilities, stakes, label=f'{n_picks} picks ({odds}x)')

ax.set_xlabel('Win Probability')
ax.set_ylabel('Kelly Stake (fraction of bankroll)')
ax.set_title('Optimal Stake Size by Probability')
ax.legend()
ax.grid(True, alpha=0.3)

# Expected value
ax = axes[0, 1]
for n_picks in [2, 3, 4, 5]:
    odds = PrizePicksBetTypes.PAYOUTS[n_picks]
    evs = [kelly.expected_value(p, odds) for p in probabilities]
    ax.plot(probabilities, evs, label=f'{n_picks} picks')

ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax.set_xlabel('Win Probability')
ax.set_ylabel('Expected Value (per $1 bet)')
ax.set_title('Expected Value by Win Probability')
ax.legend()
ax.grid(True, alpha=0.3)

# Multi-pick Kelly with correlation
ax = axes[1, 0]
base_probs = [0.6, 0.65, 0.7, 0.75]
correlations = np.linspace(0, 0.8, 20)

for n_picks in [2, 3, 4]:
    kelly_stakes = []
    for corr in correlations:
        # Adjust probabilities for correlation
        adjusted_probs = base_probs[:n_picks]
        # Simple correlation adjustment (in reality, more complex)
        combined_prob = np.prod(adjusted_probs) * (1 - corr * 0.3)
        stake = kelly.calculate_kelly_stake(combined_prob, PrizePicksBetTypes.PAYOUTS[n_picks])
        kelly_stakes.append(stake)
    
    ax.plot(correlations, kelly_stakes, label=f'{n_picks} picks')

ax.set_xlabel('Correlation Between Picks')
ax.set_ylabel('Optimal Kelly Stake')
ax.set_title('Impact of Correlation on Optimal Stake')
ax.legend()
ax.grid(True, alpha=0.3)

# Growth rate visualization
ax = axes[1, 1]
probability = 0.65
stake_fractions = np.linspace(0, 0.5, 100)

for n_picks in [2, 3, 4]:
    odds = PrizePicksBetTypes.PAYOUTS[n_picks]
    growth_rates = [kelly.calculate_growth_rate(probability, odds, f) for f in stake_fractions]
    optimal_stake = kelly.calculate_kelly_stake(probability, odds)
    
    ax.plot(stake_fractions, growth_rates, label=f'{n_picks} picks')
    ax.axvline(x=optimal_stake, color='red', linestyle='--', alpha=0.3)

ax.set_xlabel('Stake Fraction')
ax.set_ylabel('Expected Growth Rate')
ax.set_title(f'Growth Rate vs Stake Size (p={probability})')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Correlation Analysis and Management

In [ ]:
class CorrelationManager:
    """
    Manage correlations between different bet types
    """
    def __init__(self):
        # Define correlation matrix between bet types
        self.correlation_matrix = self._build_correlation_matrix()
    
    def _build_correlation_matrix(self):
        """
        Build correlation matrix between different bet types
        """
        bet_types = ['top_10', 'top_5', 'top_3', 'points', 'h2h', 
                    'beat_teammate', 'grid_gain', 'dnf']
        
        n = len(bet_types)
        corr_matrix = np.eye(n)
        
        # Define correlations (based on domain knowledge)
        correlations = {
            ('top_10', 'top_5'): 0.8,
            ('top_10', 'top_3'): 0.6,
            ('top_10', 'points'): 0.95,
            ('top_5', 'top_3'): 0.8,
            ('top_5', 'points'): 0.85,
            ('top_3', 'points'): 0.7,
            ('top_10', 'dnf'): -0.9,
            ('points', 'dnf'): -0.95,
            ('h2h', 'beat_teammate'): 0.3,
            ('grid_gain', 'top_10'): 0.2,
            ('grid_gain', 'dnf'): -0.3
        }
        
        # Build symmetric matrix
        for (type1, type2), corr in correlations.items():
            idx1 = bet_types.index(type1)
            idx2 = bet_types.index(type2)
            corr_matrix[idx1, idx2] = corr
            corr_matrix[idx2, idx1] = corr
        
        return pd.DataFrame(corr_matrix, index=bet_types, columns=bet_types)
    
    def calculate_parlay_correlation(self, bet_types, drivers=None):
        """
        Calculate overall correlation for a parlay
        """
        if len(bet_types) < 2:
            return 0
        
        total_correlation = 0
        count = 0
        
        # Pairwise correlations
        for i in range(len(bet_types)):
            for j in range(i + 1, len(bet_types)):
                bet1, bet2 = bet_types[i], bet_types[j]
                
                # Base correlation from bet types
                if bet1 in self.correlation_matrix.index and bet2 in self.correlation_matrix.index:
                    base_corr = self.correlation_matrix.loc[bet1, bet2]
                else:
                    base_corr = 0
                
                # Additional correlation if same driver
                if drivers and drivers[i] == drivers[j]:
                    base_corr = min(base_corr + 0.2, 0.95)
                
                total_correlation += abs(base_corr)
                count += 1
        
        return total_correlation / count if count > 0 else 0
    
    def adjust_probability_for_correlation(self, probabilities, correlation):
        """
        Adjust combined probability based on correlation
        """
        # Independent probability
        independent_prob = np.prod(probabilities)
        
        # Adjust for correlation (simplified model)
        # High correlation means events are more dependent
        if correlation > 0:
            # Positive correlation: if one wins, others more likely
            min_prob = min(probabilities)
            adjustment = (min_prob - independent_prob) * correlation
            adjusted_prob = independent_prob + adjustment
        else:
            # Negative correlation: if one wins, others less likely
            adjusted_prob = independent_prob * (1 + correlation * 0.5)
        
        return np.clip(adjusted_prob, 0, 1)

# Initialize correlation manager
corr_manager = CorrelationManager()

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_manager.correlation_matrix, 
            annot=True, fmt='.2f', cmap='coolwarm',
            center=0, vmin=-1, vmax=1)
plt.title('Bet Type Correlation Matrix')
plt.tight_layout()
plt.show()

# Example correlation calculations
print("\nExample Parlay Correlations:")
print("=" * 50)

examples = [
    (['top_10', 'top_5'], ['Hamilton', 'Hamilton']),
    (['top_10', 'dnf'], ['Verstappen', 'Verstappen']),
    (['top_3', 'beat_teammate'], ['Leclerc', 'Sainz']),
    (['points', 'points', 'points'], ['Hamilton', 'Verstappen', 'Leclerc'])
]

for bet_types, drivers in examples:
    corr = corr_manager.calculate_parlay_correlation(bet_types, drivers)
    print(f"Bets: {bet_types}")
    print(f"Drivers: {drivers}")
    print(f"Correlation: {corr:.3f}\n")

## 4. Prize Picks Optimization Engine

In [ ]:
class PrizePicksOptimizer:
    """
    Main optimization engine for Prize Picks selections
    """
    def __init__(self, kelly_fraction=0.25, max_correlation=0.5):
        self.kelly = KellyCriterion(kelly_fraction)
        self.corr_manager = CorrelationManager()
        self.max_correlation = max_correlation
    
    def generate_all_picks(self, predictions, min_edge=0.05):
        """
        Generate all possible picks with positive edge
        """
        picks = []
        
        for _, pred in predictions.iterrows():
            driver = pred['driver']
            
            # Check each bet type
            bet_opportunities = [
                ('top_10', pred.get('top10_prob', 0.5), 0.5),  # Implied prob
                ('top_5', pred.get('top5_prob', 0.3), 0.3),
                ('top_3', pred.get('top3_prob', 0.15), 0.15),
                ('points', pred.get('points_prob', 0.5), 0.5),
                ('beat_teammate', pred.get('beat_teammate_prob', 0.5), 0.5)
            ]
            
            for bet_type, true_prob, implied_prob in bet_opportunities:
                edge = true_prob - implied_prob
                
                if edge >= min_edge:
                    picks.append({
                        'driver': driver,
                        'bet_type': bet_type,
                        'true_prob': true_prob,
                        'implied_prob': implied_prob,
                        'edge': edge,
                        'confidence': pred.get('confidence', 0.7)
                    })
        
        return pd.DataFrame(picks)
    
    def optimize_parlay(self, available_picks, n_picks, constraints=None):
        """
        Optimize selection of n picks for a parlay
        """
        if len(available_picks) < n_picks:
            return None
        
        best_parlay = None
        best_ev = -float('inf')
        
        # Consider all combinations
        for combo in combinations(range(len(available_picks)), n_picks):
            parlay_picks = available_picks.iloc[list(combo)]
            
            # Check constraints
            if not self._check_constraints(parlay_picks, constraints):
                continue
            
            # Calculate correlation
            bet_types = parlay_picks['bet_type'].tolist()
            drivers = parlay_picks['driver'].tolist()
            correlation = self.corr_manager.calculate_parlay_correlation(bet_types, drivers)
            
            # Skip if correlation too high
            if correlation > self.max_correlation:
                continue
            
            # Calculate adjusted probability
            probs = parlay_picks['true_prob'].values
            adjusted_prob = self.corr_manager.adjust_probability_for_correlation(probs, correlation)
            
            # Calculate EV
            payout = PrizePicksBetTypes.PAYOUTS[n_picks]
            ev = self.kelly.expected_value(adjusted_prob, payout)
            
            if ev > best_ev:
                best_ev = ev
                best_parlay = {
                    'picks': parlay_picks,
                    'n_picks': n_picks,
                    'correlation': correlation,
                    'adjusted_prob': adjusted_prob,
                    'expected_value': ev,
                    'kelly_stake': self.kelly.calculate_kelly_stake(adjusted_prob, payout),
                    'payout': payout
                }
        
        return best_parlay
    
    def _check_constraints(self, picks, constraints):
        """
        Check if picks meet constraints
        """
        if not constraints:
            return True
        
        # Max picks per driver
        if 'max_per_driver' in constraints:
            driver_counts = picks['driver'].value_counts()
            if any(count > constraints['max_per_driver'] for count in driver_counts):
                return False
        
        # Max picks per bet type
        if 'max_per_type' in constraints:
            type_counts = picks['bet_type'].value_counts()
            if any(count > constraints['max_per_type'] for count in type_counts):
                return False
        
        # Minimum average edge
        if 'min_avg_edge' in constraints:
            if picks['edge'].mean() < constraints['min_avg_edge']:
                return False
        
        return True
    
    def optimize_portfolio(self, available_picks, bankroll=100, constraints=None):
        """
        Optimize entire betting portfolio across different parlay sizes
        """
        portfolio = []
        
        for n_picks in range(2, 7):  # 2-6 pick parlays
            best_parlay = self.optimize_parlay(available_picks, n_picks, constraints)
            
            if best_parlay and best_parlay['expected_value'] > 0:
                portfolio.append(best_parlay)
        
        # Allocate bankroll
        if portfolio:
            # Normalize Kelly stakes
            total_kelly = sum(p['kelly_stake'] for p in portfolio)
            
            for parlay in portfolio:
                if total_kelly > 0:
                    parlay['allocation'] = (parlay['kelly_stake'] / total_kelly) * 0.5  # Use 50% of bankroll max
                    parlay['bet_size'] = parlay['allocation'] * bankroll
                else:
                    parlay['allocation'] = 0
                    parlay['bet_size'] = 0
        
        return portfolio

# Create sample predictions for demonstration
sample_predictions = pd.DataFrame([
    {'driver': 'Verstappen', 'top10_prob': 0.85, 'top5_prob': 0.75, 'top3_prob': 0.60, 
     'points_prob': 0.85, 'beat_teammate_prob': 0.70, 'confidence': 0.85},
    {'driver': 'Hamilton', 'top10_prob': 0.80, 'top5_prob': 0.70, 'top3_prob': 0.55, 
     'points_prob': 0.80, 'beat_teammate_prob': 0.65, 'confidence': 0.80},
    {'driver': 'Leclerc', 'top10_prob': 0.75, 'top5_prob': 0.60, 'top3_prob': 0.40, 
     'points_prob': 0.75, 'beat_teammate_prob': 0.55, 'confidence': 0.75},
    {'driver': 'Perez', 'top10_prob': 0.70, 'top5_prob': 0.50, 'top3_prob': 0.30, 
     'points_prob': 0.70, 'beat_teammate_prob': 0.30, 'confidence': 0.70},
    {'driver': 'Sainz', 'top10_prob': 0.70, 'top5_prob': 0.55, 'top3_prob': 0.35, 
     'points_prob': 0.70, 'beat_teammate_prob': 0.45, 'confidence': 0.70},
    {'driver': 'Russell', 'top10_prob': 0.75, 'top5_prob': 0.60, 'top3_prob': 0.40, 
     'points_prob': 0.75, 'beat_teammate_prob': 0.35, 'confidence': 0.75}
])

# Initialize optimizer
optimizer = PrizePicksOptimizer(kelly_fraction=0.25, max_correlation=0.6)

# Generate all possible picks
all_picks = optimizer.generate_all_picks(sample_predictions, min_edge=0.05)
print(f"Generated {len(all_picks)} picks with positive edge\n")

# Optimize portfolio
constraints = {
    'max_per_driver': 2,
    'max_per_type': 3,
    'min_avg_edge': 0.10
}

portfolio = optimizer.optimize_portfolio(all_picks, bankroll=100, constraints=constraints)

print("Optimized Prize Picks Portfolio:")
print("=" * 70)
for i, parlay in enumerate(portfolio):
    print(f"\nParlay {i+1}: {parlay['n_picks']} picks")
    print(f"Expected Value: ${parlay['expected_value']:.2f} per $1")
    print(f"Win Probability: {parlay['adjusted_prob']:.1%}")
    print(f"Correlation: {parlay['correlation']:.2f}")
    print(f"Kelly Stake: {parlay['kelly_stake']:.1%}")
    print(f"Recommended Bet: ${parlay['bet_size']:.2f}")
    print("\nPicks:")
    for _, pick in parlay['picks'].iterrows():
        print(f"  - {pick['driver']} {pick['bet_type']} ({pick['true_prob']:.1%})")

## 5. Risk Management and Bankroll Optimization

In [ ]:
class RiskManager:
    """
    Risk management for Prize Picks betting
    """
    def __init__(self, max_exposure=0.25, max_single_bet=0.10):
        self.max_exposure = max_exposure  # Max % of bankroll at risk
        self.max_single_bet = max_single_bet  # Max % on single parlay
    
    def calculate_var(self, portfolio, confidence_level=0.95):
        """
        Calculate Value at Risk for portfolio
        """
        if not portfolio:
            return 0
        
        # Simulate outcomes
        n_simulations = 10000
        outcomes = []
        
        for _ in range(n_simulations):
            total_return = 0
            
            for parlay in portfolio:
                # Simulate win/loss
                if np.random.random() < parlay['adjusted_prob']:
                    # Win
                    total_return += parlay['bet_size'] * (parlay['payout'] - 1)
                else:
                    # Loss
                    total_return -= parlay['bet_size']
            
            outcomes.append(total_return)
        
        # Calculate VaR
        outcomes = np.array(outcomes)
        var = np.percentile(outcomes, (1 - confidence_level) * 100)
        
        return var
    
    def calculate_risk_metrics(self, portfolio, bankroll):
        """
        Calculate comprehensive risk metrics
        """
        total_exposure = sum(p['bet_size'] for p in portfolio)
        
        metrics = {
            'total_exposure': total_exposure,
            'exposure_pct': total_exposure / bankroll,
            'var_95': self.calculate_var(portfolio, 0.95),
            'var_99': self.calculate_var(portfolio, 0.99),
            'max_loss': -total_exposure,
            'expected_return': sum(p['expected_value'] * p['bet_size'] for p in portfolio),
            'n_bets': len(portfolio)
        }
        
        # Sharpe ratio approximation
        if metrics['total_exposure'] > 0:
            returns = [p['expected_value'] * p['bet_size'] / metrics['total_exposure'] 
                      for p in portfolio]
            metrics['sharpe_ratio'] = np.mean(returns) / (np.std(returns) + 1e-6)
        else:
            metrics['sharpe_ratio'] = 0
        
        return metrics
    
    def apply_risk_limits(self, portfolio, bankroll):
        """
        Apply risk limits to portfolio
        """
        # Check total exposure
        total_exposure = sum(p['bet_size'] for p in portfolio)
        
        if total_exposure > self.max_exposure * bankroll:
            # Scale down all bets proportionally
            scale_factor = (self.max_exposure * bankroll) / total_exposure
            for p in portfolio:
                p['bet_size'] *= scale_factor
                p['allocation'] *= scale_factor
        
        # Check individual bet limits
        for p in portfolio:
            if p['bet_size'] > self.max_single_bet * bankroll:
                p['bet_size'] = self.max_single_bet * bankroll
                p['allocation'] = p['bet_size'] / bankroll
        
        return portfolio

# Initialize risk manager
risk_manager = RiskManager(max_exposure=0.25, max_single_bet=0.10)

# Apply risk limits to portfolio
portfolio = risk_manager.apply_risk_limits(portfolio, bankroll=100)

# Calculate risk metrics
risk_metrics = risk_manager.calculate_risk_metrics(portfolio, bankroll=100)

print("\nRisk Analysis:")
print("=" * 50)
for metric, value in risk_metrics.items():
    if 'pct' in metric:
        print(f"{metric}: {value:.1%}")
    elif metric in ['var_95', 'var_99', 'max_loss', 'expected_return']:
        print(f"{metric}: ${value:.2f}")
    else:
        print(f"{metric}: {value:.2f}")

# Visualize risk profile
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Portfolio composition
ax = axes[0, 0]
bet_sizes = [p['bet_size'] for p in portfolio]
labels = [f"{p['n_picks']}-pick" for p in portfolio]
ax.pie(bet_sizes, labels=labels, autopct='%1.1f%%')
ax.set_title('Portfolio Allocation by Parlay Size')

# Expected returns distribution
ax = axes[0, 1]
n_simulations = 1000
simulation_returns = []

for _ in range(n_simulations):
    total = 0
    for p in portfolio:
        if np.random.random() < p['adjusted_prob']:
            total += p['bet_size'] * (p['payout'] - 1)
        else:
            total -= p['bet_size']
    simulation_returns.append(total)

ax.hist(simulation_returns, bins=50, alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='red', linestyle='--', label='Breakeven')
ax.axvline(x=risk_metrics['expected_return'], color='green', 
          linestyle='--', label='Expected Return')
ax.set_xlabel('Return ($)')
ax.set_ylabel('Frequency')
ax.set_title('Simulated Returns Distribution')
ax.legend()

# Risk vs Reward
ax = axes[1, 0]
for p in portfolio:
    risk = 1 - p['adjusted_prob']
    reward = p['expected_value']
    size = p['bet_size'] * 10
    ax.scatter(risk, reward, s=size, alpha=0.6, 
              label=f"{p['n_picks']}-pick")

ax.set_xlabel('Risk (Loss Probability)')
ax.set_ylabel('Expected Value per $1')
ax.set_title('Risk vs Reward by Parlay')
ax.legend()
ax.grid(True, alpha=0.3)

# Cumulative probability of profit
ax = axes[1, 1]
sorted_returns = np.sort(simulation_returns)
cumulative_prob = np.arange(1, len(sorted_returns) + 1) / len(sorted_returns)
ax.plot(sorted_returns, cumulative_prob)
ax.axvline(x=0, color='red', linestyle='--', label='Breakeven')
ax.set_xlabel('Return ($)')
ax.set_ylabel('Cumulative Probability')
ax.set_title('Cumulative Distribution of Returns')
ax.grid(True, alpha=0.3)
ax.legend()

# Add profit probability
profit_prob = (np.array(simulation_returns) > 0).mean()
ax.text(0.05, 0.95, f'P(Profit) = {profit_prob:.1%}', 
        transform=ax.transAxes, bbox=dict(boxstyle='round', facecolor='wheat'))

plt.tight_layout()
plt.show()

## 6. Real Race Application

In [ ]:
def generate_race_predictions_for_optimizer(race_name="Monaco Grand Prix"):
    """
    Generate realistic predictions for a specific race
    """
    # Driver performance profiles (simplified)
    driver_profiles = {
        'Verstappen': {'base_skill': 0.95, 'consistency': 0.90, 'monaco_bonus': 0.0},
        'Hamilton': {'base_skill': 0.90, 'consistency': 0.88, 'monaco_bonus': 0.05},
        'Leclerc': {'base_skill': 0.85, 'consistency': 0.80, 'monaco_bonus': 0.10},
        'Perez': {'base_skill': 0.82, 'consistency': 0.75, 'monaco_bonus': 0.0},
        'Sainz': {'base_skill': 0.80, 'consistency': 0.82, 'monaco_bonus': 0.05},
        'Russell': {'base_skill': 0.83, 'consistency': 0.85, 'monaco_bonus': 0.0},
        'Norris': {'base_skill': 0.78, 'consistency': 0.80, 'monaco_bonus': 0.02},
        'Piastri': {'base_skill': 0.75, 'consistency': 0.70, 'monaco_bonus': 0.0},
        'Alonso': {'base_skill': 0.77, 'consistency': 0.88, 'monaco_bonus': 0.08},
        'Ocon': {'base_skill': 0.70, 'consistency': 0.75, 'monaco_bonus': 0.03}
    }
    
    predictions = []
    
    for driver, profile in driver_profiles.items():
        # Base probabilities
        skill = profile['base_skill'] + profile.get('monaco_bonus', 0)
        
        # Calculate probabilities
        top10_prob = min(0.95, skill * 0.95)
        top5_prob = min(0.85, skill * 0.65)
        top3_prob = min(0.60, skill * 0.35)
        points_prob = top10_prob * 0.98  # Almost certain if top 10
        
        # Teammate matchups
        teammate_prob = 0.5  # Default
        if driver == 'Verstappen':
            teammate_prob = 0.75
        elif driver == 'Perez':
            teammate_prob = 0.25
        elif driver == 'Leclerc':
            teammate_prob = 0.60
        elif driver == 'Sainz':
            teammate_prob = 0.40
        elif driver == 'Hamilton':
            teammate_prob = 0.65
        elif driver == 'Russell':
            teammate_prob = 0.35
        
        predictions.append({
            'driver': driver,
            'top10_prob': top10_prob,
            'top5_prob': top5_prob,
            'top3_prob': top3_prob,
            'points_prob': points_prob,
            'beat_teammate_prob': teammate_prob,
            'confidence': profile['consistency']
        })
    
    return pd.DataFrame(predictions)

# Generate predictions for Monaco
monaco_predictions = generate_race_predictions_for_optimizer("Monaco Grand Prix")

# Run optimization
print("\n" + "=" * 70)
print("MONACO GRAND PRIX - PRIZE PICKS OPTIMIZATION")
print("=" * 70)

# Generate picks
monaco_picks = optimizer.generate_all_picks(monaco_predictions, min_edge=0.05)
print(f"\nGenerated {len(monaco_picks)} value picks")

# Optimize portfolio with specific constraints
monaco_constraints = {
    'max_per_driver': 2,  # Max 2 bets per driver
    'max_per_type': 4,    # Max 4 of same bet type
    'min_avg_edge': 0.08  # Minimum 8% edge average
}

monaco_portfolio = optimizer.optimize_portfolio(
    monaco_picks, 
    bankroll=1000,  # $1000 bankroll
    constraints=monaco_constraints
)

# Apply risk management
monaco_portfolio = risk_manager.apply_risk_limits(monaco_portfolio, bankroll=1000)

# Display optimized picks
print("\n" + "=" * 70)
print("RECOMMENDED PRIZE PICKS PORTFOLIO")
print("=" * 70)

total_bet = 0
total_expected_profit = 0

for i, parlay in enumerate(monaco_portfolio):
    print(f"\n{'='*50}")
    print(f"PARLAY {i+1}: {parlay['n_picks']}-PICK ENTRY")
    print(f"{'='*50}")
    print(f"Bet Amount: ${parlay['bet_size']:.2f}")
    print(f"Potential Payout: ${parlay['bet_size'] * parlay['payout']:.2f}")
    print(f"Win Probability: {parlay['adjusted_prob']:.1%}")
    print(f"Expected Profit: ${parlay['expected_value'] * parlay['bet_size']:.2f}")
    print(f"\nSelections:")
    
    for j, (_, pick) in enumerate(parlay['picks'].iterrows(), 1):
        print(f"  {j}. {pick['driver']} - {pick['bet_type']}")
        print(f"     Probability: {pick['true_prob']:.1%} (Line: {pick['implied_prob']:.1%})")
        print(f"     Edge: +{pick['edge']:.1%}")
    
    total_bet += parlay['bet_size']
    total_expected_profit += parlay['expected_value'] * parlay['bet_size']

# Summary
print("\n" + "=" * 70)
print("PORTFOLIO SUMMARY")
print("=" * 70)
print(f"Total Amount Wagered: ${total_bet:.2f}")
print(f"Expected Profit: ${total_expected_profit:.2f}")
print(f"Expected ROI: {(total_expected_profit/total_bet):.1%}")
print(f"Number of Parlays: {len(monaco_portfolio)}")

# Risk metrics
monaco_risk = risk_manager.calculate_risk_metrics(monaco_portfolio, bankroll=1000)
print(f"\n95% Value at Risk: ${monaco_risk['var_95']:.2f}")
print(f"Maximum Possible Loss: ${monaco_risk['max_loss']:.2f}")
print(f"Bankroll at Risk: {monaco_risk['exposure_pct']:.1%}")

## 7. Save Optimizer Configuration

In [ ]:
# Save optimizer configuration and models
optimizer_config = {
    'optimizer': optimizer,
    'risk_manager': risk_manager,
    'correlation_matrix': corr_manager.correlation_matrix,
    'payouts': PrizePicksBetTypes.PAYOUTS,
    'bet_types': PrizePicksBetTypes.BET_TYPES,
    'metadata': {
        'created_date': datetime.now().isoformat(),
        'kelly_fraction': optimizer.kelly.kelly_fraction,
        'max_correlation': optimizer.max_correlation,
        'risk_limits': {
            'max_exposure': risk_manager.max_exposure,
            'max_single_bet': risk_manager.max_single_bet
        }
    }
}

# Save configuration
joblib.dump(optimizer_config, 'f1_prize_picks_optimizer.pkl')
print("\nPrize Picks Optimizer saved successfully!")
print(f"Configuration: {optimizer_config['metadata']}")

## Summary

The Prize Picks Optimizer provides:

1. **Kelly Criterion**: Optimal bet sizing based on edge and bankroll
2. **Correlation Management**: Accounts for dependencies between bets
3. **Portfolio Optimization**: Selects best combination of parlays
4. **Risk Management**: VaR calculations and exposure limits
5. **Expected Value Maximization**: Focus on long-term profitability

### Key Features:
- Automatically identifies value bets (positive edge)
- Optimizes across different parlay sizes (2-6 picks)
- Manages correlation to avoid over-concentration
- Applies conservative Kelly sizing (25% fraction)
- Provides detailed explanations for each pick

### Usage Guidelines:
- Only bet with positive expected value
- Diversify across drivers and bet types
- Monitor actual results vs predictions
- Adjust edge requirements based on confidence
- Never risk more than 25% of bankroll